In [36]:
import pandas as pd
import os
import librosa

audio_dataset_path=r"C:\\Users\\asus\\Downloads\\seg_data16sr\\seg_data16sr\\all"
metadata=pd.read_csv (r"C:\\Users\\asus\\Downloads\\minor code\\coughdataset\\newmfcc2.csv")
metadata.head()

,FileName,Status
0,positive (1).wav,Covid
1,positive (10).wav,Covid
2,positive (100).wav,Covid
3,positive (1000).wav,Covid
4,positive (1001).wav,Covid


In [37]:

def features_extractor(file):
    audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
    
    return mfccs_scaled_features

In [38]:

import numpy as np
from tqdm import tqdm
### Now we iterate through every audio file and extract features 
### using Mel-Frequency Cepstral Coefficients
extracted_features=[]
for index_num,row in tqdm(metadata.iterrows()):
    file_name = os.path.join(os.path.abspath(audio_dataset_path),str(row["FileName"]))
    final_class_labels=row["Status"]
    data=features_extractor(file_name)
    extracted_features.append([data,final_class_labels])

1882it [01:32, 20.37it/s]


In [39]:
### converting extracted_features to Pandas dataframe
extracted_features_df=pd.DataFrame(extracted_features,columns=['feature','corona_test'])
extracted_features_df.head()

,feature,corona_test
0,"[-1109.2283, 5.6909633, -4.3914094, 2.92813, -...",Covid
1,"[-601.4879, 62.74279, -71.83314, 86.28924, -49...",Covid
2,"[-643.25684, 124.871086, -53.615955, 41.385952...",Covid
3,"[-364.84286, 68.06273, -4.579537, 93.7206, -29...",Covid
4,"[-544.8134, 69.37885, 2.8263512, 66.01771, -2....",Covid


In [74]:
class_names = extracted_features_df.corona_test

In [72]:

### Split the dataset into independent and dependent dataset
X=np.array(extracted_features_df['feature'].tolist())
y=np.array(extracted_features_df['corona_test'].tolist())

In [70]:
y

array(['Covid', 'Covid', 'Covid', ..., 'Non-Covid', 'Non-Covid',
       'Non-Covid'], dtype='<U9')

In [41]:

### Label Encoding
y=np.array(pd.get_dummies(y))

In [42]:
### Train Test Split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [43]:

import tensorflow as tf
print(tf.__version__)

2.4.1


In [44]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam
from sklearn import metrics

In [45]:
### No of classes
num_labels=y.shape[1]

In [46]:

model=Sequential()
###first layer
model.add(Dense(100,input_shape=(40,)))
model.add(Activation('relu'))
model.add(Dropout(0.1))
###second layer
model.add(Dense(200))
model.add(Activation('relu'))
model.add(Dropout(0.1))
###third layer
model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.1))

###final layer
model.add(Dense(num_labels))
model.add(Activation('softmax'))

In [47]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 100)               4100      
_________________________________________________________________
activation_4 (Activation)    (None, 100)               0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 200)               20200     
_________________________________________________________________
activation_5 (Activation)    (None, 200)               0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 100)              

In [48]:
model.compile(loss='binary_crossentropy',metrics=['accuracy'],optimizer='adam')

In [49]:
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime 

num_epochs = 100
num_batch_size = 38

checkpointer = ModelCheckpoint(filepath='saved_models/audio_classification.hdf5', 
                               verbose=1, save_best_only=True)
start = datetime.now()

clf=model.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(X_test, y_test), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

Epoch 1/100
40/40 [==============================] - 2s 15ms/step - loss: 4.5797 - accuracy: 0.6044 - val_loss: 0.8855 - val_accuracy: 0.6790

Epoch 00001: val_loss improved from inf to 0.88554, saving model to saved_models\audio_classification.hdf5
Epoch 2/100
40/40 [==============================] - 0s 7ms/step - loss: 1.2197 - accuracy: 0.6120 - val_loss: 0.6086 - val_accuracy: 0.6923

Epoch 00002: val_loss improved from 0.88554 to 0.60858, saving model to saved_models\audio_classification.hdf5
Epoch 3/100
40/40 [==============================] - 0s 8ms/step - loss: 0.7124 - accuracy: 0.6735 - val_loss: 0.6066 - val_accuracy: 0.7188

Epoch 00003: val_loss improved from 0.60858 to 0.60664, saving model to saved_models\audio_classification.hdf5
Epoch 4/100
40/40 [==============================] - 0s 7ms/step - loss: 0.6737 - accuracy: 0.6787 - val_loss: 0.6117 - val_accuracy: 0.7135

Epoch 00004: val_loss did not improve from 0.60664
Epoch 5/100
40/40 [==============================] 

40/40 [==============================] - 0s 7ms/step - loss: 0.4034 - accuracy: 0.8074 - val_loss: 0.5053 - val_accuracy: 0.7772

Epoch 00038: val_loss did not improve from 0.47785
Epoch 39/100
40/40 [==============================] - 0s 7ms/step - loss: 0.4111 - accuracy: 0.8077 - val_loss: 0.4852 - val_accuracy: 0.7613

Epoch 00039: val_loss did not improve from 0.47785
Epoch 40/100
40/40 [==============================] - 0s 7ms/step - loss: 0.4012 - accuracy: 0.8051 - val_loss: 0.4996 - val_accuracy: 0.7613

Epoch 00040: val_loss did not improve from 0.47785
Epoch 41/100
40/40 [==============================] - 0s 7ms/step - loss: 0.3677 - accuracy: 0.8253 - val_loss: 0.4886 - val_accuracy: 0.7586

Epoch 00041: val_loss did not improve from 0.47785
Epoch 42/100
40/40 [==============================] - 0s 6ms/step - loss: 0.3786 - accuracy: 0.8334 - val_loss: 0.4881 - val_accuracy: 0.7772

Epoch 00042: val_loss did not improve from 0.47785
Epoch 43/100
40/40 [=======================


Epoch 00079: val_loss did not improve from 0.47156
Epoch 80/100
40/40 [==============================] - 0s 7ms/step - loss: 0.2260 - accuracy: 0.8950 - val_loss: 0.5380 - val_accuracy: 0.7851

Epoch 00080: val_loss did not improve from 0.47156
Epoch 81/100
40/40 [==============================] - 0s 6ms/step - loss: 0.2162 - accuracy: 0.9084 - val_loss: 0.5169 - val_accuracy: 0.8090

Epoch 00081: val_loss did not improve from 0.47156
Epoch 82/100
40/40 [==============================] - 0s 8ms/step - loss: 0.2037 - accuracy: 0.9034 - val_loss: 0.5119 - val_accuracy: 0.8143

Epoch 00082: val_loss did not improve from 0.47156
Epoch 83/100
40/40 [==============================] - 0s 7ms/step - loss: 0.2233 - accuracy: 0.8957 - val_loss: 0.5504 - val_accuracy: 0.7958

Epoch 00083: val_loss did not improve from 0.47156
Epoch 84/100
40/40 [==============================] - 0s 7ms/step - loss: 0.2110 - accuracy: 0.9071 - val_loss: 0.5636 - val_accuracy: 0.8223

Epoch 00084: val_loss did not

In [50]:
test_accuracy=model.evaluate(X_test,y_test,verbose=0)
print(test_accuracy[1])

0.8328912258148193


In [51]:
y_pred=model.predict(X_test)



In [52]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test.argmax(axis=1),y_pred.argmax(axis=1))


array([[244,  22],
       [ 41,  70]], dtype=int64)

In [32]:
import matplotlib.pyplot as plt
from sklearn.metrics import plot_confusion_matrix

In [60]:
cm=metrics.confusion_matrix(y_test.argmax(axis=1),y_pred.argmax(axis=1))
plt.imshow(cm)


In [34]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test.argmax(axis=1),y_pred.argmax(axis=1)))

0.8355437665782494


In [35]:
from sklearn.metrics import classification_report
print(classification_report(y_test.argmax(axis=1),y_pred.argmax(axis=1)))


              precision    recall  f1-score   support

           0       0.88      0.89      0.88       266
           1       0.72      0.71      0.72       111

    accuracy                           0.84       377
   macro avg       0.80      0.80      0.80       377
weighted avg       0.83      0.84      0.84       377



In [66]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
cnf_matrix = confusion_matrix(y_test.argmax(axis=1),y_pred.argmax(axis=1))
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names,
                      title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True,
                      title='Normalized confusion matrix')

plt.show()

Confusion matrix, without normalization
[[244  22]
 [ 41  70]]
Normalized confusion matrix
[[0.92 0.08]
 [0.37 0.63]]
